In [ ]:
# code sketch:
# 1. small subset of songs, get lyrics / info from spotify / genius api
#     turn lyrics coupled w metadata into vector using BERT or something
#.    store in Pinecone? FAISS
# 2. user input > gpt 4 / langchain to get semantic vector rep. of that
#.     compare using cosine sim., closest 10 get suggested


In [1]:
# import all necessary files
from setup import *
from spotify_handler import *
from genius_handler import *
#from langchain_handler import *

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
songNumber =  10 # number of songs we're gonna sample from randomly
pbar = tqdm(total = songNumber) #progress bar init
query_offset = 998 # number of songs to offset the search by
max_workers = 8 # number of workers to use for the multithreading

build_track_list(songNumber, query_offset, max_workers)
songs = read_from_sql()

                                      
                                      
  0%|          | 0/10 [00:02<?, ?it/s]        

1
2


                                      
  0%|          | 0/10 [00:02<?, ?it/s]        

3
No lyrics found for Zeppelin See by Montbretia.


                                      
  0%|          | 0/10 [00:04<?, ?it/s]        

...

                                      
  0%|          | 0/10 [00:21<?, ?it/s]        

9


                                      
  0%|          | 0/10 [00:22<?, ?it/s]        

10


100%|██████████| 10/10 [00:25<00:00,  2.53s/it]


In [3]:
#print(len(trackList[0]))
#print(trackList)
print(songs)

#songs.to_csv("songs.csv", sep="|")

      index                                             Lyrics  \
0         0  51 ContributorsTranslationsPortuguêsEspañolTür...   
1         1  98 ContributorsTranslationsTürkçeEspañolDeutsc...   
2         2  38 ContributorsLet’s Stay Together Lyrics[Intr...   
3         3  2 ContributorsUpgrade U (Beyoncé Experience Li...   
4         4  109 ContributorsTranslationsEspañol​hot girl b...   
...     ...                                                ...   
2245      5  18 ContributorsTranslationsEspañolTürkçeLike C...   
2246      6  21 ContributorsKilby Girl Lyrics[Verse 1]\nWe'...   
2247      7  22 ContributorsPeople I Don’t Like Lyrics[Chor...   
2248      8  390 ContributorsTranslationsEspañolРусскийEngl...   
2249      9  122 ContributorsChill Bill Lyrics[Intro: Rob $...   

                                 Song Name               Artist  Danceability  \
0                          I’m Good (Blue)         David Guetta         0.561   
1                                The Light   

In [ ]:
#LANGCHAIN TIME
# here gonna take each lyric file and summarize it / vectorize it and store in pinecone
# initialize langchain / openAI

docs=[]
summary = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".txt"):  # check if the file is a .txt file
        file_path = os.path.join(directory, filename)
        # now you can do something with the file
        loader = TextLoader(file_path)
        doc = loader.load()
        docs.append(doc)
for i in tqdm(docs):
    summary.append(chain.run(i))

        

In [ ]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='asia-northeast1-gcp'
)

index_name="spotifysemantic"

for i in tqdm(docs):
    docsearch=Pinecone.from_documents(i, embeddings, index_name=index_name)





In [ ]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "songs for the best day of my life"
ans = docsearch.similarity_search(query)

for i in ans:
    print(i.metadata)